In [1]:
import pandas as pd
import numpy as np

In [2]:
dogs = pd.read_csv('dogs.csv')
dogs.head()

,HALTER_ID,ALTER,GESCHLECHT,STADTKREIS,STADTQUARTIER,RASSE1,RASSE1_MISCHLING,RASSE2,RASSE2_MISCHLING,RASSENTYP,GEBURTSJAHR_HUND,GESCHLECHT_HUND,HUNDEFARBE
0,126,51-60,m,9.0,92.0,Welsh Terrier,NaN,NaN,NaN,K,2011,w,schwarz/braun
1,574,61-70,w,2.0,23.0,Cairn Terrier,NaN,NaN,NaN,K,2002,w,brindle
2,695,41-50,m,6.0,63.0,Labrador Retriever,NaN,NaN,NaN,I,2012,w,braun
3,893,61-70,w,7.0,71.0,Mittelschnauzer,NaN,NaN,NaN,I,2010,w,schwarz
4,1177,51-60,m,10.0,102.0,Shih Tzu,NaN,NaN,NaN,K,2011,m,schwarz/weiss


# data cleanup

## translate to english

In [3]:
dogs.rename(columns={'HALTER_ID': 'Owner_ID', 'ALTER': 'Owner_Age', 'GESCHLECHT': 'Owner_Gender',
                     'STADTKREIS': 'City_District', 'STADTQUARTIER': 'City_Quarter',
                     'RASSE1': 'Primary_Breed', 'RASSE1_MISCHLING': 'Primary_Breed_Hybrid',
                     'RASSE2': 'Secondary_Breed', 'RASSE2_MISCHLING': 'Secondary_Breed_Hybrid',
                     'RASSENTYP': 'Breed_Type', 'GEBURTSJAHR_HUND': 'Dog_Birth_Year',
                     'GESCHLECHT_HUND': 'Dog_Gender', 'HUNDEFARBE': 'Dog_Color'}, inplace=True)

,Owner_ID,Owner_Age,Owner_Gender,City_District,City_Quarter,Primary_Breed,Primary_Breed_Hybrid,Secondary_Breed,Secondary_Breed_Hybrid,Breed_Type,Dog_Birth_Year,Dog_Gender,Dog_Color
0,126,51-60,m,9.0,92.0,Welsh Terrier,NaN,NaN,NaN,K,2011,w,schwarz/braun
1,574,61-70,w,2.0,23.0,Cairn Terrier,NaN,NaN,NaN,K,2002,w,brindle
2,695,41-50,m,6.0,63.0,Labrador Retriever,NaN,NaN,NaN,I,2012,w,braun
3,893,61-70,w,7.0,71.0,Mittelschnauzer,NaN,NaN,NaN,I,2010,w,schwarz
4,1177,51-60,m,10.0,102.0,Shih Tzu,NaN,NaN,NaN,K,2011,m,schwarz/weiss
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6975,128471,21-30,w,8.0,83.0,Tibetan Spaniel,NaN,NaN,NaN,K,2007,w,braun/beige
6976,128471,21-30,w,8.0,83.0,Rottweiler,NaN,NaN,NaN,I,2014,m,schwarz/rot
6977,128479,51-60,w,6.0,63.0,Lagotto Romagnolo,Mischling,NaN,NaN,I,2007,m,weiss/grau
6978,128482,21-30,w,9.0,91.0,Yorkshire Terrier,NaN,NaN,NaN,K,2015,w,schwarz/braun/grau


## check for null

In [4]:
dogs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6980 entries, 0 to 6979
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Owner_ID                6980 non-null   int64  
 1   Owner_Age               6978 non-null   object 
 2   Owner_Gender            6980 non-null   object 
 3   City_District           6978 non-null   float64
 4   City_Quarter            6978 non-null   float64
 5   Primary_Breed           6980 non-null   object 
 6   Primary_Breed_Hybrid    668 non-null    object 
 7   Secondary_Breed         537 non-null    object 
 8   Secondary_Breed_Hybrid  0 non-null      float64
 9   Breed_Type              6806 non-null   object 
 10  Dog_Birth_Year          6980 non-null   int64  
 11  Dog_Gender              6980 non-null   object 
 12  Dog_Color               6980 non-null   object 
dtypes: float64(3), int64(2), object(8)
memory usage: 709.0+ KB


### drop unnecessary columns

In [5]:
null_columns = ['Primary_Breed_Hybrid', 'Secondary_Breed', 'Secondary_Breed_Hybrid']
dogs = dogs.drop(columns=null_columns, axis=1)
dogs.head()

,Owner_ID,Owner_Age,Owner_Gender,City_District,City_Quarter,Primary_Breed,Breed_Type,Dog_Birth_Year,Dog_Gender,Dog_Color
0,126,51-60,m,9.0,92.0,Welsh Terrier,K,2011,w,schwarz/braun
1,574,61-70,w,2.0,23.0,Cairn Terrier,K,2002,w,brindle
2,695,41-50,m,6.0,63.0,Labrador Retriever,I,2012,w,braun
3,893,61-70,w,7.0,71.0,Mittelschnauzer,I,2010,w,schwarz
4,1177,51-60,m,10.0,102.0,Shih Tzu,K,2011,m,schwarz/weiss


### check for data entry errors, or values that don't make sense

In [6]:
dogs['Dog_Birth_Year'].max()

5012

In [7]:
dogs.loc[dogs['Dog_Birth_Year'] == 5012]

,Owner_ID,Owner_Age,Owner_Gender,City_District,City_Quarter,Primary_Breed,Breed_Type,Dog_Birth_Year,Dog_Gender,Dog_Color
4543,111954,41-50,w,3.0,34.0,Parson Russell Terrier,K,5012,m,tricolor


In [11]:
dogs = dogs.drop(dogs.index[[4543]])
dogs['Dog_Birth_Year'].max()  # good

2016

In [17]:
dogs['Dog_Birth_Year'].min()

1962

In [13]:
dogs.loc[dogs['Dog_Birth_Year'] == 1]

,Owner_ID,Owner_Age,Owner_Gender,City_District,City_Quarter,Primary_Breed,Breed_Type,Dog_Birth_Year,Dog_Gender,Dog_Color
3165,98411,71-80,w,11.0,119.0,Labrador Retriever,I,1,w,schwarz


In [14]:
dogs = dogs.drop(dogs.index[[3165]])
dogs['Dog_Birth_Year'].min()

8

In [15]:
dogs.loc[dogs['Dog_Birth_Year'] == 8]

,Owner_ID,Owner_Age,Owner_Gender,City_District,City_Quarter,Primary_Breed,Breed_Type,Dog_Birth_Year,Dog_Gender,Dog_Color
2901,94655,41-50,w,2.0,24.0,Labrador Retriever,I,8,w,braun


In [16]:
dogs = dogs.drop(dogs.index[[2901]])
dogs['Dog_Birth_Year'].min()  # good

1962

# 1. Does city region affect average size or quantity of dog(s) owned?

## Join with weight category data

In [18]:
dogs_weight = pd.read_csv('dogs_weight.csv')
dogs_weight.head()

,name_in_data,translation,weight_category,frequency
0,Mischling klein,mixed breed small,11-20,652
1,Chihuahua,chihuahua,1-10,495
2,Labrador Retriever,labrador retriever,71-80,446
3,Jack Russel Terrier,jack russell terrier,11-20,298
4,Yorkshire Terrier,yorkshire terrier,1-10,280


In [20]:
dogs_weight[['weight_lower_bound', 'weight_upper_bound']] = dogs_weight['weight_category'].str.split('-', expand=True)
dogs_weight.head()

,name_in_data,translation,weight_category,frequency,weight_lower_bound,weight_upper_bound
0,Mischling klein,mixed breed small,11-20,652,11,20
1,Chihuahua,chihuahua,1-10,495,1,10
2,Labrador Retriever,labrador retriever,71-80,446,71,80
3,Jack Russel Terrier,jack russell terrier,11-20,298,11,20
4,Yorkshire Terrier,yorkshire terrier,1-10,280,1,10


In [28]:
dogs_weight = dogs_weight.rename({'name_in_data': 'Primary_Breed'}, axis=1)
dogs_weight.head(1)

,Primary_Breed,translation,weight_lower_bound,weight_upper_bound
0,Mischling klein,mixed breed small,11,20


### manipulation of columns

In [24]:
dogs_weight = dogs_weight.drop(['weight_category', 'frequency'], axis=1)
dogs_weight.head()

,name_in_data,translation,weight_lower_bound,weight_upper_bound
0,Mischling klein,mixed breed small,11,20
1,Chihuahua,chihuahua,1,10
2,Labrador Retriever,labrador retriever,71,80
3,Jack Russel Terrier,jack russell terrier,11,20
4,Yorkshire Terrier,yorkshire terrier,1,10


In [30]:
joined_dogs = pd.merge(dogs, dogs_weight, on='Primary_Breed')
joined_dogs.tail()

,Owner_ID,Owner_Age,Owner_Gender,City_District,City_Quarter,Primary_Breed,Breed_Type,Dog_Birth_Year,Dog_Gender,Dog_Color,translation,weight_lower_bound,weight_upper_bound
6970,126607,51-60,m,11.0,115.0,Nackthund,K,2012,w,schwarz,Xoloitzcuintle,41,50
6971,126732,41-50,w,3.0,31.0,Australien Kelpie,I,2015,w,braun,Australian Kelpie,31,40
6972,126947,31-40,w,11.0,119.0,Chinese Powder Puff,K,2010,m,weiss,Chinese Crested dog,1,10
6973,128403,61-70,m,4.0,42.0,Wolfspitz,K,2014,m,tricolor,keeshond,31,40
6974,128406,31-40,w,7.0,74.0,Wäller,I,2015,w,fauve,waller,51,60


# 2. Are most dog owners in Zurich repeat dog owners? Do the repeat owners get smaller dogs as they get older?

# 3. Are most dogs in Zurich mixed breed or purebred? Why?